# Results from MILP Realtime Computations

In [1]:
import numpy as np
import pandas as pd

from ctmmodels.const import *

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

# Varying parameters: demand, alpha, beta, gamma

## Saving dataframes

In [4]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Loading CSV files

In [5]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
}

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [6]:
_results_dflist = []

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_old.csv'.format(demand))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_old.csv'.format(demand))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_old.csv'.format(demand))
        else:
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e3 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch3_a{}_b{}_c{}.csv'.format(demand, *weights))

        df_e1 = df_e1.rename(columns=_col_rename)
        df_e1['demand'] = [demand]
        df_e1['new_model'] = weights != (0,0,0)
        df_e1['alpha'] = weights[0]
        df_e1['beta'] = weights[1]
        df_e1['gamma'] = weights[2]
        df_e1['model_type'] = _model_type[weights]
        df_e1['epoch'] = 1
        
        df_e2 = df_e2.rename(columns=_col_rename)
        df_e2['demand'] = [demand]
        df_e2['new_model'] = weights != (0,0,0)
        df_e2['alpha'] = weights[0]
        df_e2['beta'] = weights[1]
        df_e2['gamma'] = weights[2]
        df_e2['model_type'] = _model_type[weights]
        df_e2['epoch'] = 2
        
        df_e3 = df_e3.rename(columns=_col_rename)
        df_e3['demand'] = [demand]
        df_e3['new_model'] = weights != (0,0,0)
        df_e3['alpha'] = weights[0]
        df_e3['beta'] = weights[1]
        df_e3['gamma'] = weights[2]
        df_e3['model_type'] = _model_type[weights]
        df_e3['epoch'] = 3
        
        _results_dflist.append(pd.concat([df_e1, df_e2, df_e3]))

In [7]:
df = pd.concat(_results_dflist)

## Adding initial MILP results

In [8]:
df_initial_results = pd.read_pickle(DF_PATH + 'results.pkl')

In [9]:
df_initial_results.head()

,demand_ns,demand_ew,runtime,delay,throughput,objective_value,alpha,beta,gamma
0,450,450,746.312135,895.0,77.0,413.828019,1.0,0.0,0.0
1,450,450,258.463889,1170.5,79.5,-297.372159,0.0,1.0,0.0
2,450,450,596.386801,1106.5,76.5,-2243.750000,0.0,0.0,1.0
3,450,450,1129.751537,895.0,77.0,62.903593,0.5,0.5,0.0
4,450,450,426.153587,1059.0,79.0,-1268.584280,0.0,0.5,0.5


In [10]:
df.head()

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,1.003851,8510.430380,67.835443,3935.033010,450,True,1.0,0.0,0.0,Delay priority,1
0,1.791379,12507.974683,67.835443,5783.408250,450,True,1.0,0.0,0.0,Delay priority,2
0,1.824722,16820.962025,67.835443,7777.637309,450,True,1.0,0.0,0.0,Delay priority,3
0,2.755098,8538.506329,66.974684,-250.520833,450,True,0.0,1.0,0.0,Throughput priority,1
0,0.300883,11532.607595,68.835443,-257.481061,450,True,0.0,1.0,0.0,Throughput priority,2


In [11]:
df_initial_results['demand'] = df_initial_results.apply(lambda row: 
                                                        int(row['demand_ns']) if row['demand_ns'] == row['demand_ew']
                                                        else (int(row['demand_ns']), int(row['demand_ew']))
                                                        , axis=1)

df_initial_results['new_model'] = df_initial_results.apply(lambda row: row['alpha'] != 0, axis=1)

df_initial_results['model_type'] = df_initial_results.apply(lambda row: _model_type[
    (row['alpha'], row['beta'], row['gamma'])
], axis=1)

df_initial_results['epoch'] = 0

df_initial_results['delay'] = df_initial_results['delay'].apply(lambda x: 2*x)

In [12]:
df_initial_results = df_initial_results[[
    'runtime',
    'delay',
    'throughput',
    'objective_value',
    'demand',
    'new_model',
    'alpha',
    'beta',
    'gamma',
    'model_type',
    'epoch'
]]

In [13]:
df = df.rename(columns={'obj-value': 'objective_value'})

In [15]:
df_final = pd.concat([df_initial_results, df]).sort_values(by=['epoch', 'demand', 'model_type'])

In [16]:
save_df(df_final, 'milp-realtime-results')

In [17]:
df_final

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,746.312135,1790.000000,77.000000,413.828019,450,True,1.00,0.00,0.00,Delay priority,0
5,1736.445266,1826.000000,76.000000,-909.757925,450,True,0.50,0.00,0.50,Delay-Flow priority,0
3,1129.751537,1790.000000,77.000000,62.903593,450,True,0.50,0.50,0.00,Delay-Throughput priority,0
6,1369.602678,1844.000000,79.500000,-696.512281,450,True,0.33,0.33,0.33,Equal priority,0
2,596.386801,2213.000000,76.500000,-2243.750000,450,False,0.00,0.00,1.00,Flow priority,0
7,301.049237,2313.000000,79.500000,79.500000,450,False,0.00,0.00,0.00,Parent model,0
1,258.463889,2341.000000,79.500000,-297.372159,450,False,0.00,1.00,0.00,Throughput priority,0
4,426.153587,2118.000000,79.000000,-1268.584280,450,False,0.00,0.50,0.50,Throughput-Flow priority,0
8,133.554586,6036.303797,92.278481,1380.209780,900,True,1.00,0.00,0.00,Delay priority,0
13,353.227471,6134.354160,96.208861,-1309.774565,900,True,0.50,0.00,0.50,Delay-Flow priority,0
